In [7]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [9]:
data_dir = "/content/drive/MyDrive/Animal Classification/dataset"

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=30,
    zoom_range=0.3,
    shear_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

train_gen = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_gen = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)
import os
for folder in os.listdir(data_dir):
    path = os.path.join(data_dir, folder)
    if os.path.isdir(path):
        print(f"{folder}: {len(os.listdir(path))} images")


Found 1561 images belonging to 15 classes.
Found 383 images belonging to 15 classes.
Bear: 125 images
Bird: 137 images
Cat: 123 images
Cow: 131 images
Deer: 127 images
Dog: 122 images
Dolphin: 129 images
Elephant: 133 images
Giraffe: 129 images
Horse: 130 images
Kangaroo: 126 images
Lion: 131 images
Panda: 135 images
Tiger: 129 images
Zebra: 137 images


In [10]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load base model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base model layers

# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.4)(x)
x = Dense(256, activation='relu')(x)
output = Dense(train_gen.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,589,775 (9.88 MB)

 Trainable params: 331,791 (1.27 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [11]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ModelCheckpoint("/content/drive/MyDrive/Animal Classification/best_model.h5", save_best_only=True)
]

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,
    callbacks=callbacks
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 12s/step - accuracy: 0.0865 - loss: 3.2715 

49/49 ━━━━━━━━━━━━━━━━━━━━ 721s 15s/step - accuracy: 0.0872 - loss: 3.2640 - val_accuracy: 0.3394 - val_loss: 2.2023
Epoch 2/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2913 - loss: 2.3042

49/49 ━━━━━━━━━━━━━━━━━━━━ 134s 3s/step - accuracy: 0.2920 - loss: 2.3015 - val_accuracy: 0.5901 - val_loss: 1.6969
Epoch 3/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5004 - loss: 1.7544

49/49 ━━━━━━━━━━━━━━━━━━━━ 117s 2s/step - accuracy: 0.5011 - loss: 1.7529 - val_accuracy: 0.6815 - val_loss: 1.3044
Epoch 4/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6125 - loss: 1.4167

...

49/49 ━━━━━━━━━━━━━━━━━━━━ 116s 2s/step - accuracy: 0.7745 - loss: 0.8031 - val_accuracy: 0.7624 - val_loss: 0.7742
Epoch 9/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7814 - loss: 0.7532

49/49 ━━━━━━━━━━━━━━━━━━━━ 116s 2s/step - accuracy: 0.7815 - loss: 0.7526 - val_accuracy: 0.7937 - val_loss: 0.6835
Epoch 10/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 113s 2s/step - accuracy: 0.7876 - loss: 0.7070 - val_accuracy: 0.7911 - val_loss: 0.6955
Epoch 11/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 141s 2s/step - accuracy: 0.7966 - loss: 0.6506 - val_accuracy: 0.7885 - val_loss: 0.6894
Epoch 12/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8170 - loss: 0.6069

49/49 ━━━━━━━━━━━━━━━━━━━━ 115s 2s/step - accuracy: 0.8170 - loss: 0.6073 - val_accuracy: 0.7990 - val_loss: 0.6492
Epoch 13/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 133s 3s/step - accuracy: 0.8273 - loss: 0.5717 - val_accuracy: 0.7755 - val_loss: 0.7137
Epoch 14/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8250 - loss: 0.5661

49/49 ━━━━━━━━━━━━━━━━━━━━ 133s 3s/step - accuracy: 0.8250 - loss: 0.5662 - val_accuracy: 0.8016 - val_loss: 0.6434
Epoch 15/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 132s 3s/step - accuracy: 0.8344 - loss: 0.5449 - val_accuracy: 0.7937 - val_loss: 0.6456
Epoch 16/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8567 - loss: 0.4862

49/49 ━━━━━━━━━━━━━━━━━━━━ 114s 2s/step - accuracy: 0.8565 - loss: 0.4868 - val_accuracy: 0.8277 - val_loss: 0.5680
Epoch 17/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 131s 3s/step - accuracy: 0.8652 - loss: 0.4855 - val_accuracy: 0.8094 - val_loss: 0.5706
Epoch 18/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 113s 2s/step - accuracy: 0.8608 - loss: 0.4790 - val_accuracy: 0.7911 - val_loss: 0.6323
Epoch 19/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 141s 2s/step - accuracy: 0.8580 - loss: 0.4709 - val_accuracy: 0.7990 - val_loss: 0.6054


In [12]:
# Save mapping of index to class
class_indices = train_gen.class_indices
index_to_class = {v: k for k, v in class_indices.items()}
print(index_to_class)


{0: 'Bear', 1: 'Bird', 2: 'Cat', 3: 'Cow', 4: 'Deer', 5: 'Dog', 6: 'Dolphin', 7: 'Elephant', 8: 'Giraffe', 9: 'Horse', 10: 'Kangaroo', 11: 'Lion', 12: 'Panda', 13: 'Tiger', 14: 'Zebra'}


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

img_path = "/content/drive/MyDrive/Animal Classification/dataset/Zebra/your_image.jpg"  # Change path
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

prediction = model.predict(img_array)
predicted_class = index_to_class[np.argmax(prediction)]
print(f"Predicted class: {predicted_class}")


In [13]:
from google.colab import files
uploaded = files.upload()


Saving elephant.jpg to elephant.jpg


In [15]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the uploaded image
img = image.load_img("elephant.jpg", target_size=(224, 224))  # Replace with your image name

# Convert to array and normalize
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)  # Shape becomes (1, 224, 224, 3)


In [18]:
# Predict
prediction = model.predict(img_array)

# Map index to class name
predicted_class = index_to_class[np.argmax(prediction)]
print(f"Predicted animal: {predicted_class}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
Predicted animal: Elephant
